In [ ]:
# ler todos os arquivos csv do diretório e guardar em um objeto
# import required modules
import numpy as np
import time
import os
import re
import pandas as pd
from dask import dataframe as dd
from collections import Counter
from tqdm import tqdm
diretorio = r'Bases/Arquivos_Limpos/'
all_files = list(filter(lambda x: '.csv' in x, os.listdir(diretorio)))

#Warnings: Possui uma série de funções e comandos para tratamento de mensagens de avisos e alertas do Python
import warnings
warnings.filterwarnings("ignore")

#Comando para exibir todas colunas do arquivo
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Layout CNPJ
CNPJ = {"EMPRE" : ['CNPJ_BASE', 'RAZAO_SOCIAL', 'NATUREZA_JURIDICA' , 'QUALIFICACAO' ,
          'VALOR_CAPITAL_SOCIAL' , 'PORTE_EMPRESA'  , 'ENTE_FEDERATIVO'],

"ESTABELE" : ['CNPJ_BASE', 'CNPJ_ORDEM' , 'CNPJ_DV' , 'MATRIZ_FILIAL'  , 'NOME_FANTASIA' , 'SITUACAO_CADASTRAL' ,
              'DATA_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  , 'CIDADE_EXTERIOR'  , 'PAIS'  , 'DATA_INICIO_ATIVIDADE'  ,
              'CNAE_PRINCIPAL'  , 'CNAE_SECUNDARIO' , 'TIPO_LOGRADOURO'  , 'LOGRADOURO'  , 'NUMERO'  , 'COMPLEMENTO' ,
              'BAIRRO'  , 'CEP'  , 'UF'  , 'MUNICIPIO'  , 'DDD1'  , 'TELEFONE1'  , 'DDD2'  , 'TELEFONE2'  ,
              'DDD_FAX'  , 'FAX'  , 'EMAIL'  , 'SITUACAO_ESPECIAL'  , 'DATA_SITUACAO_ESPECIAL'],  

"SIMPLES" : ['CNPJ_BASE'  , 'OPCAO_SIMPLES'  , 'DATA_OPCAO_SIMPLES'  , 'DATA_EXCLUSAO_SIMPLES'  ,
             'OPCAO_MEI'  , 'DATA_OPCAO_MEI'  , 'DATA_EXCLUSAO_MEI'],

"SOCIO" : ['CNPJ_BASE'  , 'TIPO'  , 'NOME'  , 'CPF_CNPJ'  , 'QUALIFICACAO'  , 'DATA_ENTRADA' ,
            'PAIS' , 'REPRESENTANTE'  , 'NOME_REPRESENTANTE'  , 'QUALIFICACAO_REPRESENTANTE'  , 'FAIXA_ETARIA'  ],

"PAIS" : ['CODIGO_PAIS' , 'PAIS'],

"MUNIC" : ['MUNICIPIO'  , 'MUNICIPIO'  ],

"QUALS" : ['QUALIFICACAO', 'QUALIFICACAO' ],

"NATJU" : ['NATUREZA_JURIDICA'  , 'NATUREZA_JURIDICA'],

"MOTI" : ['MOTIVO_SITUACAO_CADASTRAL'  , 'MOTIVO_SITUACAO_CADASTRAL'  ],

"CNAE" : ['CODIGO_CNAE'  , 'CNAE' ]}


In [2]:
cartoes = pd.read_excel(r"C:\Users\ABRASEL NACIONAL\Documents\CNPJ_PROGRAMATICA\Painel_Leads\BaseCsv\ATUALIZADO_December_Estabelecimentos_BenVisaVale.xlsx" , usecols=[0,1])
cartoes.head(5)


,Estabelecimento,Endereço
0,GRAN PASTEL GOURMET,"RUA CARACA 934, 934 CASA NOSSA SENHORA DA CONC..."
1,M130 SERRA,"R IVAI, 130 SERRA, BELO HORIZONTE - MG"
2,COMBATE DISTRIBUIDORA,"RUA BAMBUI 56, 56 LOJA A NOSSA SENHORA DA CONC..."
3,RESTAURANTE GFH 8 EIRELI,"RUA ORIENTE 827, 2323 PORTARIA - SERRA, BELO H..."
4,EPA SUPERMERCADOS,"R CARACA, 600 SERRA, BELO HORIZONTE - MG"


In [3]:
lista_lojas_cartoes = cartoes['Estabelecimento'].map(str).to_list()
lista_enderecos_cartoes = cartoes['Endereço'].map(str).to_list()
lista_enderecos_cartoes = str(lista_enderecos_cartoes)
del cartoes

In [14]:
for i in lista_lojas_cartoes:
    if i == 'POINT BEER':
        print(i)
    else:
        continue

POINT BEER


In [4]:
df_1 = pd.read_csv(f'{diretorio}{all_files[0]}', names=['NOME_FANTASIA','LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'], sep=';',encoding='utf-8', usecols=[4, 14, 15, 16, 17], low_memory=False)
df_1['LOGRADOURO_NUMERO'] = df_1['LOGRADOURO'].map(str) + ' ' + df_1['NUMERO'].map(str)
df_1['LOGRADOURO_NUMERO'].replace(regex='nan', value='', inplace=True)
df_1['COMPLEMENTO_BAIRRO'] = df_1['COMPLEMENTO'].map(str) + ' ' + df_1['BAIRRO'].map(str)
df_1['COMPLEMENTO_BAIRRO'].replace(regex='nan', value='', inplace=True)
df_1.drop(['LOGRADOURO','NUMERO','COMPLEMENTO','BAIRRO'],axis=1, inplace=True)
df_1.head(5)


,NOME_FANTASIA,LOGRADOURO_NUMERO,COMPLEMENTO_BAIRRO
0,AGO RESTOBAR,CLOUTIDES PEREIRA ROCHA SN,BAIRRO CEARENSE
1,Indisponível,GETULIO VARGAS 158,CENTRO
2,PROMESSA DE DEUS,SALAO 681,BLOCO CASA RUA AUGUSTINHO ALEGRE CHIC
3,PESQUE & PAGUE BOM PESQUEIRO,RODOVIA PR364 SN,CABECEIRA DO ARROIO GRANDE
4,BAR DO BEM,PARA 440,PACOVAL


In [20]:
NOMEBEN_OK = []
for row in tqdm(df_1['NOME_FANTASIA']):
    if row in lista_lojas_cartoes:
        NOMEBEN_OK.append(row[1])
    else:
        NOMEBEN_OK.append(0)
print(Counter(NOMEBEN_OK).keys())
print(Counter(NOMEBEN_OK).values())

25534it [00:50, 497.58it/s]

In [7]:
RUA_NUMERO_BEN_OK = []
for row in tqdm(df_1['LOGRADOURO_NUMERO']):
    if row in lista_enderecos_cartoes:
        RUA_NUMERO_BEN_OK.append(1)
    else:
        RUA_NUMERO_BEN_OK.append(0)
print(Counter(RUA_NUMERO_BEN_OK).keys())
print(Counter(RUA_NUMERO_BEN_OK).values())

74905it [02:45, 451.97it/s]

dict_keys([0, 1])
dict_values([72339, 2566])


In [5]:
COMPLEMENTO_BAIRRO_BEN_OK = []
for row in tqdm(df_1['COMPLEMENTO_BAIRRO']):
    if row in lista_enderecos_cartoes:
        COMPLEMENTO_BAIRRO_BEN_OK.append(1)
    else:
        COMPLEMENTO_BAIRRO_BEN_OK.append(0)
print(Counter(COMPLEMENTO_BAIRRO_BEN_OK).keys())
print(Counter(COMPLEMENTO_BAIRRO_BEN_OK).values())

74905it [01:49, 683.72it/s]

dict_keys([0, 1])
dict_values([38942, 35963])


In [13]:
aceitaben = { 'NOMEBEN_OK': NOMEBEN_OK, 'RUA_NUMERO_BEN_OK': RUA_NUMERO_BEN_OK , 'COMPLEMENTO_BAIRRO_BEN_OK' : COMPLEMENTO_BAIRRO_BEN_OK}
dados = pd.DataFrame(aceitaben)
dados.head(5)

,NOMEBEN_OK,RUA_NUMERO_BEN_OK,COMPLEMENTO_BAIRRO_BEN_OK
0,0,0,0
1,0,0,1
2,0,0,0
3,0,0,0
4,0,0,1
